#### Algorithm.
Raytracing problem. One of the hardest one till now.



In [2]:
""" Challenge 4.1. Raytracing problem """

import collections as col
from math import sqrt, atan2

class LaserFight:
    def __init__(self, dim, player_pos, guard_pos, dist):
        self.room, self.max_dist = tuple(dim), dist
        self.player, self.guard = tuple(player_pos), tuple(guard_pos)
        self.polar_coord, self.cart_coord = col.defaultdict(None, { }), col.defaultdict(None, { })
    
    """
    Subroutine for generating reflections...
    Reflections of the point (x, y) ==>
        Over the line x = a: (-x + 2*a, y),   a in R
        Over the line y = b: (x, -y + 2*b),   b in R
        Special case: a = 0 ==> y-axis, b = 0 ==> x-axis
    """
    def Reflection(self, point, a = None, b = None):
        x, y = tuple(point)
        if (a == None) and (b == None):    return (x, y)
        elif (a != None) and (b == None):  return (-x + 2*a, y)
        elif (a == None) and (b != None):  return (x, -y + 2*b)
        else:                              return (-x + 2*a, -y + 2*b)
    
    ## Check if the reflected point is valid and store corresponding coordinates...
    def IsValid(self, point, origin = None):
        origin = self.player if (origin == None) else origin        # All calculations are wrt player
        x, y = tuple(map(lambda p1, p2: p1 - p2, point, origin))
        r, theta = sqrt(x**2 + y**2), atan2(y, x)
        self.polar_coord[tuple(point)] = (r, theta)
        self.cart_coord[(r, theta)] = tuple(point)
        return (0 < r <= self.max_dist)
    
    def GetReflections(self):
        ## Number of times to mirror across each dimension...
        self.mirrors = tuple(map(lambda pp, rr: (pp + self.max_dist) // rr + 1, self.player, self.room))
        
        ## Positive reflections...
        guard_refl_x,  guard_refl_y  = col.deque([self.guard]), col.deque()
        player_refl_x, player_refl_y = col.deque([self.player]), col.deque()
        for i in range(self.mirrors[0]):                       # Reflect point over x = a
            gg, pp = guard_refl_x[-1], player_refl_x[-1]
            if i > 0:
                gg = self.Reflection(gg, a = self.room[0]*i)
                pp = self.Reflection(pp, a = self.room[0]*i)
            guard_refl_x.append(gg) if self.IsValid(gg) else None
            player_refl_x.append(pp) if self.IsValid(pp) else None 
            for j in range(1, self.mirrors[1]):                # Reflect point over y = a
                gg = self.Reflection(gg, b = self.room[1]*j)
                pp = self.Reflection(pp, b = self.room[1]*j)
                guard_refl_y.append(tuple(gg)) if self.IsValid(gg) else None
                player_refl_y.append(tuple(pp)) if self.IsValid(pp) else None
        ## End of nested loops ##
        
        ## Combine reflection sets over x = a and y = b...
        guard_refl = set(guard_refl_x) | set(guard_refl_y)
        player_refl = set(player_refl_x) | set(player_refl_y)
        
        ## Negative reflections...
        ##     Reflect each point over x = 0, y = 0, & (0, 0)...
        guard_refl_neg = col.deque()
        for gg in guard_refl:
            gg_neg = [self.Reflection(gg, a = 0), self.Reflection(gg, b = 0), self.Reflection(gg, a = 0, b = 0)]
            [guard_refl_neg.append(gn) for gn in gg_neg if self.IsValid(gn)];
        
        player_refl_neg = col.deque()
        for pp in player_refl:
            pp_neg = [self.Reflection(pp, a = 0), self.Reflection(pp, b = 0), self.Reflection(pp, a = 0, b = 0)]
            [player_refl_neg.append(pn) for pn in pp_neg if self.IsValid(pn)]
        
        ## Combine all reflections...
        guard_refl  |= set(guard_refl_neg);        player_refl |= set(player_refl_neg)
        player_refl.remove(self.player)                # Remove original player position
        return (guard_refl, player_refl)  

def solution(dimensions, your_position, guard_position, distance):
    player, guard = your_position, guard_position        # Rename to shorted variables
    fight = LaserFight(dimensions, player, guard, distance)
    guard_pos_all, player_pos_all = fight.GetReflections()
    
    ## Get all player positions at different angles with original player position...
    player_pos = col.defaultdict(None, {})
    for pp in player_pos_all:
        r, theta = fight.polar_coord[pp]
        cond1 = (0 < r <= fight.max_dist)
        cond2 = (theta not in player_pos) or ((theta in player_pos) and (r < player_pos[theta][0]))
        if (cond1 and cond2):    player_pos[theta] = (r, theta)
    ## End of loop ##
    
    ## Get all guard positions at different angles with original player position...
    guard_pos, beam_bearings = col.defaultdict(None, {}), col.defaultdict(None, {})
    for gg in guard_pos_all:
        r, theta = fight.polar_coord[gg]
        cond1 = (0 < r <= fight.max_dist)
        cond2 = (theta not in guard_pos) or ((theta in guard_pos) and (r < guard_pos[theta][0]))
        cond3 = (theta not in player_pos) or ((theta in player_pos) and (r < player_pos[theta][0]))
        if (cond1 and cond2 and cond3):
            guard_pos[theta] = (r, theta)
            beam_bearings[theta] = (gg[0] - player[0], gg[1] - player[1])
    ## End of loop ##
    
    beam_bearings = list(beam_bearings.values())
    beam_count = len(beam_bearings)
    return beam_count


In [17]:
## Examples...
%time print("Number of ways to hit the guard = ",\
            solution(dimensions = [3, 2], your_position = [1, 1], guard_position = [2, 1], distance = 4))
%time print("Number of ways to hit the guard = ",\
            solution(dimensions = [300, 275], your_position = [150, 150], guard_position = [185, 100], distance = 500))
%time print("Number of ways to hit the guard = ",\
            solution(dimensions = [2, 5], your_position = [1, 2], guard_position = [1, 4], distance = 11))
%time print("Number of ways to hit the guard = ",\
            solution(dimensions = [23, 10], your_position = [6, 4], guard_position = [3, 2], distance = 23))
%time print("Number of ways to hit the guard = ",\
            solution(dimensions = [1250, 1250], your_position = [1000, 1000], guard_position = [500, 400], distance = 10000))
%time print("Number of ways to hit the guard = ",\
            solution(dimensions = [10, 10], your_position = [4, 4], guard_position = [3, 3], distance = 5000))


Number of ways to hit the guard =  7
Wall time: 998 µs
Number of ways to hit the guard =  9
Wall time: 0 ns
Number of ways to hit the guard =  27
Wall time: 1.99 ms
Number of ways to hit the guard =  8
Wall time: 1e+03 µs
Number of ways to hit the guard =  196
Wall time: 3.99 ms
Number of ways to hit the guard =  739323
Wall time: 16.7 s
